<a href="https://colab.research.google.com/github/NeuroWinter/news-headline-ai/blob/master/Spacy_transfomers_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install spacy-transformers
!python3 -m spacy download en_trf_distilbertbaseuncased_lg
!pip install torch

     |████████████████████████████████| 61kB 3.1MB/s 
     |████████████████████████████████| 10.4MB 9.9MB/s 
     |████████████████████████████████| 296kB 70.6MB/s 
     |████████████████████████████████| 61kB 11.2MB/s 
     |████████████████████████████████| 122kB 68.7MB/s 
     |████████████████████████████████| 3.7MB 66.9MB/s 
     |████████████████████████████████| 2.2MB 66.7MB/s 
     |████████████████████████████████| 1.0MB 63.8MB/s 
     |████████████████████████████████| 870kB 55.5MB/s 
  Created wheel for spacy-transformers: filename=spacy_transformers-0.5.1-py2.py3-none-any.whl size=52837 sha256=247f5248d56ae6e0b2e3ba1a9131b9cc3f1bc64332539ce30bff6534c5a99a8e
  Stored in directory: /root/.cache/pip/wheels/29/c2/17/625a3d14da8cabe9781ab1648d489d1b41a8a81dc289e5af1f
  Created wheel for torchcontrib: filename=torchcontrib-0.0.2-cp36-none-any.whl size=7530 sha256=d54c9744276ae5fb8e0e7938bdb4f10c97f3e153f0663df5dcf41049239aeccd
  Stored in directory: /root/.cache/pip/wheels/06/06

In [0]:
import pandas as pd
import csv

# REMEBER TO UPLOAD THIS!
input_file = 'news_headlines.csv'

TRAIN_DATA = []
df = pd.read_csv(input_file)

list_of_sites = list(df['news_site'].unique())

for row in df.iterrows():
    row_headline       = row[1][2]
    row_cat            = {site: 0 for site in list_of_sites}
    row_cat[row[1][1]] += 1
    TRAIN_DATA.append((row_headline.strip(), {"site": row_cat}))
print(TRAIN_DATA[0])

('Experts say adapting to climate change can pay off manifold', {'site': {'ABC - Tech': 1, 'ABC - Top Stories': 0, 'ABC - US': 0, 'ABC - World': 0, 'Al Jazeera': 0, 'AllAfrica News - Mozambique': 0, 'ARRL': 0, 'Ars Technica - all features': 0, 'Ars Technica - all news': 0, 'Axios - Business': 0, 'Axios - Energy': 0, 'Axios - Future': 0, 'Axios - Health Care': 0, 'Axios - Politics': 0, 'Axios - Science': 0, 'Axios - Technology': 0, 'Axios - Top': 0, 'Axios - World': 0, 'Bandladesh - bdnews24.com': 0, 'Bangkok Post - Lifestyle': 0, 'Bangkok Post - Most Recent': 0, 'Bangkok Post - Top Stories': 0, 'Bangkok Post - Travel': 0, 'BBC': 0, 'BBC - Africa': 0, 'BBC - Asia': 0, 'BBC - England': 0, 'BBC - Europe': 0, 'BBC - Latin America': 0, 'BBC - Middle East': 0, 'BBC - Northern Ireland': 0, 'BBC - Scotland': 0, 'BBC - UK': 0, 'BBC - US & Canada': 0, 'BBC - Wales': 0, 'BBC - World': 0, 'Breitbart "News"': 0, 'Buzzfeed - US News': 0, 'CanLii - Regulations': 0, 'CanLii - Supreme Court of Canada':

In [0]:
# RUN THIS TO MAKE SURE THAT SPACY IS WORKING
import spacy

nlp = spacy.load('en_trf_distilbertbaseuncased_lg')
doc = nlp("Apple shares rose on the news. Apple pie is delicious.")
print(doc[0].similarity(doc[7]))
print(doc._.trf_last_hidden_state.shape)

0.7756554
(16, 768)


In [0]:
TRAIN_DATA = [
    ("text1", {"cats": {"POSITIVE": 1.0, "NEGATIVE": 0.0}})
]

In [0]:
import spacy
from spacy.util import minibatch
import random
import torch

BATCH_SIZE = 1000

is_using_gpu = spacy.prefer_gpu()
if is_using_gpu:
    torch.set_default_tensor_type("torch.cuda.FloatTensor")

nlp = spacy.load("en_trf_distilbertbaseuncased_lg")
print(nlp.pipe_names) # ["sentencizer", "trf_wordpiecer", "trf_tok2vec"]
textnews = nlp.create_pipe("trf_textcat", config={"exclusive_classes": True})
for label in list_of_sites:
    textnews.add_label(label)
nlp.add_pipe(textnews)
optimizer = nlp.resume_training()
for i in range(10):
    random.shuffle(TRAIN_DATA)
    losses = {}
    for x, batch in enumerate(minibatch(TRAIN_DATA, size=BATCH_SIZE)):
        print(x)
        texts, news = zip(*batch)
        nlp.update(texts, news, sgd=optimizer, losses=losses)
    print(i, losses)

['sentencizer', 'trf_wordpiecer', 'trf_tok2vec']
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
0 {'trf_textcat': nan}
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27


In [0]:
a = nlp("RANDOM TEXT")
a.cats

{'NEGATIVE': 0.00117073196452111, 'POSITIVE': 0.9988293051719666}